<center>
<h1>CODE DE WEB SCRAPING</h1>
<h2>Plateforme scrapée : glassdoor</h2>
<h3>Par : ATOUKOUVI Komi Dénis</h2>
</center>

In [33]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import time
import os
import random
import logging

## Configuration du service et des options

In [34]:
service = Service("chromedriver.exe")

# Configuration des options de Chrome

options = Options()
options.add_argument("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36")
options.add_argument("--disable-blink-features=AutomationControlled")
# options.add_argument("--headless")

In [35]:
# Configuration d'un ficher de suivi des erreurs
logging.basicConfig(filename='scraping_errors.log', level=logging.ERROR)

## Gestion des URLs de façon dynamique

In [36]:
# URL de base et paramètres de recherche d'emploi
base_url = "https://www.glassdoor.com/Job/france-{}-jobs-SRCH_IL.0,6_IN86_KO7,32.htm"

job_types = {
    "ingénieur-en-informatique": {"CDI": "5QWDV", "CDD": "T9BXE"},
    "ingénieur-en-intelligence-artificielle": {"CDI": "5QWDV", "CDD": "T9BXE"},
    "ingénieur-devops": {"CDI": "5QWDV", "CDD": "T9BXE"},
    "ingénieur-cybersécurité": {"CDI": "5QWDV", "CDD": "T9BXE"},
    "data-engineer": {"CDI": "5QWDV", "CDD": "T9BXE"},
    "cloud-computing-engineer": {"CDI": "5QWDV", "CDD": "T9BXE"},
    "data-scientist": {"CDI": "5QWDV", "CDD": "T9BXE"},
    "ingénieur-énérgies-renouvelables": {"CDI": "5QWDV", "CDD": "T9BXE"},
    "ingénieur-électrotechnique": {"CDI": "5QWDV", "CDD": "T9BXE"},
    "ingénieur-génie-civil": {"CDI": "5QWDV", "CDD": "T9BXE"},
    "ingénieur-robotique": {"CDI": "5QWDV", "CDD": "T9BXE"},
    "ingénieur-mécanique": {"CDI": "5QWDV", "CDD": "T9BXE"},
    "ingénieur-industriel": {"CDI": "5QWDV", "CDD": "T9BXE"},
    "ingénieur-automobile": {"CDI": "5QWDV", "CDD": "T9BXE"},
    "ingénieur-télécommunications": {"CDI": "5QWDV", "CDD": "T9BXE"},
    "ingénieur-électronique": {"CDI": "5QWDV", "CDD": "T9BXE"},
    "ingénieur-financier": {"CDI": "5QWDV", "CDD": "T9BXE"},
    "ingénieur-électronique": {"CDI": "5QWDV", "CDD": "T9BXE"},
    "ingénieur-financier": {"CDI": "5QWDV", "CDD": "T9BXE"},
    "ingénieur-aéronautique": {"CDI": "5QWDV", "CDD": "T9BXE"},
    "ingénieur-data-scientist": {"CDI": "5QWDV", "CDD": "T9BXE"},
    "ingénieur-logiciel": {"CDI": "5QWDV", "CDD": "T9BXE"},
    "ingénieur-full-stack": {"CDI": "5QWDV", "CDD": "T9BXE"},
    "ingénieur-réseaux-sécurité": {"CDI": "5QWDV", "CDD": "T9BXE"},
    "ingénieur-civil": {"CDI": "5QWDV", "CDD": "T9BXE"}
}

In [37]:
# Fonction pour générer les URLs de façon dynamique

def generate_urls(base_url, job_types):
    urls = []
    for job_title, types in job_types.items():
        for contract_type, job_code in types.items():
            url = f"{base_url.format(job_title)}?jobTypeIndeed={job_code}"
            urls.append(url)
    return urls

urls = generate_urls(base_url, job_types)

In [38]:
# Fichier pour sauvegarder les données

file_name = 'glassdoor_jobs_dataset.csv'

# Fonction d'nitialisation du driver

def init_browser():
    return webdriver.Chrome(service=service, options=options)

# Fonction pour détecter le type de contract à travers l'URL

def detect_contract_type(url):
    if "5QWDV" in url:
        return "CDI"
    elif "T9BXE" in url:
        return "CDD"
    else:
        return "Non spécifié"

## Fonction pour scrapper à partir d'un URL

In [39]:
## Fonction pour scraper un url

def scrape_data_from_url(driver, url):
    driver.get(url)
    time.sleep(random.uniform(5, 10))  # Pause pour éviter d'être détecté
    try:
        # Attendre que les données principales soient chargées
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'JobCard_jobCardContainer__arQlW'))
        )
    except Exception as e:
        print(f"Data not loaded properly for {url}: {e}")
    # Récupérer le contenu de la page après l'attente
    page_content = driver.page_source
    page_content = driver.page_source
    soup = BeautifulSoup(page_content, 'html.parser')
    
    # Détecter automatiquement le type de contrat
    contract_type = detect_contract_type(url)
    
    job_listings = []
    for job_card in soup.find_all('div', class_='JobCard_jobCardContainer__arQlW'):
        title = job_card.find('a', class_='JobCard_jobTitle__GLyJ1').get_text(strip=True) if job_card.find('a', class_='JobCard_jobTitle__GLyJ1') else "Non spécifié"
        company = job_card.find('span', class_='EmployerProfile_compactEmployerName__9MGcV').get_text(strip=True) if job_card.find('span', class_='EmployerProfile_compactEmployerName__9MGcV') else "Non spécifié"
        location = job_card.find('div', class_='JobCard_location__Ds1fM').get_text(strip=True) if job_card.find('div', class_='JobCard_location__Ds1fM') else "Non spécifié"
        salary = job_card.find('div', class_='JobCard_salaryEstimate__QpbTW').get_text(strip=True) if job_card.find('div', class_='JobCard_salaryEstimate__QpbTW') else "Non spécifié"
        rating = job_card.find('span', class_='rating-single-star_RatingText__XENmU').get_text(strip=True) if job_card.find('span', class_='rating-single-star_RatingText__XENmU') else "Non spécifié"

        job_listings.append({
            'Titre': title,
            'Entreprise': company,
            'Localisation': location,
            'Salaire': salary,
            'Contrat': contract_type,
            'Note_entreprise' : rating
        })
    
    return job_listings


## Principale fonction de scraping en se servant de la précédente fonction

In [40]:
# Principale Fonction de scraping (itération sur tous les URLs)

def main_scrape(urls):
    all_job_listings = []
    
    for url in urls:
        try:
            # Initialisation du navigateur pour chaque URL
            driver = init_browser()
            print(f"Scraping URL: {url}")

            # Scraper les données à partir de l'URL actuel
            job_data = scrape_data_from_url(driver, url)
            all_job_listings.extend(job_data)
            print(f"Data scraped for this URL")
            
            # Fermer le navigateur après chaque URL
            driver.quit()
            
            # Pause après chaque URL pour éviter d'être détecter
            time.sleep(random.uniform(5, 15))
        
        except Exception as e:
            # Charger les erreurs et fermer le navigateur
            logging.error(f"Failed to scrape {url}: {e}")
            driver.quit()
            time.sleep(random.uniform(5, 10))
    
    return all_job_listings


## Sauvegarde des données en .csv

In [41]:
# Sauvegarder les données en csv

def save_to_csv(data, file_name):
    df = pd.DataFrame(data)
    if os.path.exists(file_name):
        df.to_csv(file_name, mode='a', header=False, index=False, encoding='utf-8')
        print(f"Data appended to {file_name}")
    else:
        df.to_csv(file_name, index=False, encoding='utf-8')
        print(f"All Data saved to new file {file_name}")

## SCRAPING

In [42]:
# Scraper tous les URLs et sauvegarder les données

all_job_listings = main_scrape(urls)
save_to_csv(all_job_listings, file_name)

Scraping URL: https://www.glassdoor.com/Job/france-ingénieur-en-informatique-jobs-SRCH_IL.0,6_IN86_KO7,32.htm?jobTypeIndeed=5QWDV
Data not loaded properly for https://www.glassdoor.com/Job/france-ingénieur-en-informatique-jobs-SRCH_IL.0,6_IN86_KO7,32.htm?jobTypeIndeed=5QWDV: name 'WebDriverWait' is not defined
Data scraped for this URL
Scraping URL: https://www.glassdoor.com/Job/france-ingénieur-en-informatique-jobs-SRCH_IL.0,6_IN86_KO7,32.htm?jobTypeIndeed=T9BXE
Data not loaded properly for https://www.glassdoor.com/Job/france-ingénieur-en-informatique-jobs-SRCH_IL.0,6_IN86_KO7,32.htm?jobTypeIndeed=T9BXE: name 'WebDriverWait' is not defined
Data scraped for this URL
Scraping URL: https://www.glassdoor.com/Job/france-ingénieur-en-intelligence-artificielle-jobs-SRCH_IL.0,6_IN86_KO7,32.htm?jobTypeIndeed=5QWDV
Data not loaded properly for https://www.glassdoor.com/Job/france-ingénieur-en-intelligence-artificielle-jobs-SRCH_IL.0,6_IN86_KO7,32.htm?jobTypeIndeed=5QWDV: name 'WebDriverWait' i